In [1]:
import numpy as np
from utils.slam_helpers import transform_to_frame,transformed_params2depthplussilhouette,transformed_params2rendervar,transformed_GRNparams2rendervar,transformed_GRNparams2depthplussilhouette
from diff_gaussian_rasterization import GaussianRasterizer as Renderer
from scripts.main_SurgeSplat import deform_gaussians, setup_camera
import torch
import os
from PIL import Image

System Paths:
/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM
/media/thesis_ssd/anaconda/envs/endogslam/lib/python310.zip
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/lib-dynload

/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/site-packages
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/site-packages/setuptools/_vendor
/tmp/tmp9wrqhw11


In [2]:
intrinsics = torch.tensor([[199.6883,   0.0000, 166.3290],
        [  0.0000, 249.4753, 170.4058],
        [  0.0000,   0.0000,   1.0000]], device='cuda:0')
w2c = torch.tensor([[ 1.0000e+00,  6.5711e-11,  2.3283e-10,  0.0000e+00],
        [-3.1832e-11,  1.0000e+00, -7.4115e-21,  0.0000e+00],
        [-9.2644e-22,  2.9104e-11,  1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')

cam = setup_camera(336,336, intrinsics.cpu().numpy(), w2c.detach().cpu().numpy(), use_simplification=True,bg = [1,1,1])




In [3]:
exp = 1
all_params = np.load(f'/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments_backup/RARP simple_3/simple_3/params.npz',allow_pickle=True)
params={}
for key in all_params.keys():
    try:
        params[key] = torch.tensor(all_params[key]).cuda()
    except:
        params[key] = [torch.tensor(all_params[key][i]).cuda() for i in range(all_params[key].shape[0])]

In [4]:
# try:
#     all_params = {k: torch.tensor(all_params[k]).cuda().float() for k in all_params.keys()}


#     keys = [k for k in all_params.keys() if
#             k not in ['org_width', 'org_height', 'w2c', 'intrinsics', 
#                     'gt_w2c_all_frames', 'cam_unnorm_rots',
#                     'cam_trans', 'keyframe_time_indices']]

#     params = all_params
#     for k in keys:
#         if not isinstance(all_params[k], torch.Tensor):
#             params[k] = torch.tensor(all_params[k]).cuda().float()
#         else:
#             params[k] = all_params[k].cuda().float()
# except:
#     # keys = [k for k in all_params.keys() if
#     #         k not in ['org_width', 'org_height', 'w2c', 'intrinsics', 
#     #                 'gt_w2c_all_frames', 'cam_unnorm_rots',
#     #                 'cam_trans', 'keyframe_time_indices']]
#     params={}
#     for key in all_params.keys():
#         try:
#             params[key] = torch.tensor(all_params[key]).cuda()
#         except:
#             params[key] = [torch.tensor(all_params[key][i]).cuda() for i in range(all_params[key].shape[0])]

In [5]:
params['means3D'][-1].shape

torch.Size([114920, 3])

In [6]:
def deform_gaussians(params, time, deform_grad, N=5,deformation_type = 'gaussian'):
    """
    Calculate deformations using the N closest basis functions based on |time - bias|.

    Args:
        params (dict): Dictionary containing deformation parameters.
        time (torch.Tensor): Current time step.
        deform_grad (bool): Whether to calculate gradients for deformations.
        N (int): Number of closest basis functions to consider.

    Returns:
        xyz (torch.Tensor): Updated 3D positions.
        rots (torch.Tensor): Updated rotations.
        scales (torch.Tensor): Updated scales.
    """
    if deformation_type =='gaussian':
        if True:
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0)

            # Apply the mask to weights and biases
            masked_weights = weights * mask
            masked_biases = biases * mask

            # Calculate deformations
            deform = torch.sum(
                masked_weights * torch.exp(-1 / (2 * stds**2) * (time - masked_biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]
        else:
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0).detach()

            # Register a gradient hook to zero out gradients for irrelevant basis functions
            if deform_grad:
                def zero_out_irrelevant_gradients(grad):
                    return grad * mask

                weights.register_hook(zero_out_irrelevant_gradients)
                biases.register_hook(zero_out_irrelevant_gradients)
                stds.register_hook(zero_out_irrelevant_gradients)

            # Calculate deformations
            deform = torch.sum(
                weights * torch.exp(-1 / (2 * stds**2) * (time - biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]

        xyz = params['means3D'] + deform_xyz
        rots = params['unnorm_rotations'] + deform_rots
        scales = params['log_scales'] + deform_scales
        opacities = params['logit_opacities']
        colors = params['rgb_colors']


    elif deformation_type == 'simple':
        # with torch.no_grad():
        xyz = params['means3D'][time]
        rots = params['unnorm_rotations'][time]
        scales = params['log_scales'][time]
        opacities = params['logit_opacities'][time]
        colors = params['rgb_colors'][time]

    return xyz, rots, scales,opacities, colors
for i in range(params['cam_trans'].shape[-1]):
    params['cam_trans'][...,i][...,-1] += 1

for id in range(params['cam_unnorm_rots'].shape[-1]):
    local_means,local_rots,local_scales,local_opacities,local_colors = deform_gaussians(params,id,deform_grad = True,deformation_type='simple')


    #  print(torch.sum(local_means-params['means3D']))

    transformed_pts = transform_to_frame(local_means,params,id,False,False)





    # Initialize Render Variables
    rendervar = transformed_GRNparams2rendervar(params, transformed_pts,local_rots,local_scales,local_opacities,local_colors)
    print(local_scales.max())
    rv_store = {}
    for key in rendervar.keys():
        rv_store[key] = rendervar[key].cpu().detach()
        local_means_store = local_means.cpu()
        local_scales_store = local_rots.cpu()
        local_rots_store = local_rots.cpu()
        transformed_pts_store = transformed_pts.cpu()



    #  rendervar['means3D'].retain_grad()

    depth_sil_rendervar = transformed_GRNparams2depthplussilhouette(params, w2c,
                                            transformed_pts,local_rots,local_scales,local_opacities)


    #RGB Rendering

    rendervar['means2D'].retain_grad()
    im, radius, _ = Renderer(raster_settings=cam)(**rendervar)
    # variables['means2D'] = rendervar['means2D'] # Gradient only accum from colour render for densification
    img = Image.fromarray((im.permute(1,2,0).cpu().detach().numpy()*255).astype(np.uint8))
    os.makedirs(f'./eval_plots/plots_simple/',exist_ok=True)
    img.save(f'./eval_plots/plots_simple/{id}.png')

    print(local_means.shape)


tensor(91.0990, device='cuda:0')
torch.Size([112733, 3])
tensor(91.0990, device='cuda:0')
torch.Size([112733, 3])
tensor(91.0990, device='cuda:0')
torch.Size([112749, 3])
tensor(91.0990, device='cuda:0')
torch.Size([112759, 3])
tensor(91.0990, device='cuda:0')
torch.Size([112769, 3])
tensor(91.0027, device='cuda:0')
torch.Size([112780, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112808, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112822, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112822, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112897, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112951, 3])
tensor(90.9985, device='cuda:0')
torch.Size([112992, 3])
tensor(90.9985, device='cuda:0')
torch.Size([113020, 3])
tensor(90.9985, device='cuda:0')
torch.Size([113035, 3])
tensor(90.9985, device='cuda:0')
torch.Size([113047, 3])
tensor(90.9985, device='cuda:0')
torch.Size([113055, 3])
tensor(90.9985, device='cuda:0')
torch.Size([113055, 3])
tensor(90.9985, device='cuda:0'

In [6]:
params['cam_trans'].shape

torch.Size([1, 3, 409])

In [7]:
# import matplotlib.pyplot as plt
# weights =   params['deform_weights'].cpu().detach()
# biases =    params['deform_biases'].cpu().detach()
# stds =      params['deform_stds'].cpu().detach()

# deforms = []
# deformsx1 = []
# for time in range(100):
#     deform = torch.sum(weights*torch.exp(-1/(2*stds**2)*(time-biases)**2),1)
#     deforms.append(deform) # Nx10 gaussians deformations
#     deformsx1.append(deform[0,0])
# fig,ax = plt.subplots(10,10,figsize = (25,25),sharey=True)
# for i in range(10):
#     for ii in range(10):
#         ax[ii,i].plot([deforms[idx][i+91720//3,ii] for idx in range(len(deforms))])
#         ax[ii,i].axhline(y = 0.0,color = 'r', linestyle = '--')    

In [8]:
# from GRN.models.conv_unet import GaussianRegressionNetwork
# import torch

# state_dict = torch.load('logs/GRN_6/checkpoint.pth',weights_only=False)
# model_state_dict = state_dict['model']

# model = GaussianRegressionNetwork()
# # model.load_state_dict(model_state_dict)

# torch.save(model.state_dict(),'GRN/models/GRN_v1.pth')


In [9]:
# import numpy as np
# import matplotlib.pyplot as plt
# depth = np.load('/media/thesis_ssd/data/Endonerf/cutting_tissues_twice-20241203T074853Z-001/cutting_tissues_twice/depth/depth_frame-000000.npz')
# depth = depth['data']
# plt.imshow(depth)
# plt.colorbar()

In [10]:
# import os
# import cv2
# import matplotlib.pyplot as plt
# image_paths = os.listdir('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/data/RARP/frames/')
# # image_paths = image_paths[:2]
# for image_path in image_paths:
#     image = cv2.imread(os.path.join('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/data/RARP/frames/', image_path))
#     # Convert to grayscale
#     image_cropped = image[75:-75, 300:-300]

#     cv2.imwrite(os.path.join('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/data/RARP/frames', f'cropped_{image_path}'), image_cropped)


In [5]:
image_paths[0].replace('EndoNerf ','')

'cutting_SurgeDepth'

In [78]:
import os
image_paths = os.listdir('experiments')
image_paths = [os.path.join('experiments',image_path,image_path.replace("EndoNerf ", '')) for image_path in image_paths]
image_paths

['experiments/EndoNerf cutting_SurgeDepth/cutting_SurgeDepth',
 'experiments/EndoNerf cutting_SurgeDepth_GRN/cutting_SurgeDepth_GRN',
 'experiments/EndoNerf cutting_SurgeDepth_GRN_10_percent/cutting_SurgeDepth_GRN_10_percent',
 'experiments/Stereomis_2/Stereomis_2',
 'experiments/EndoNerf cutting_SurgeDepth_random_init/cutting_SurgeDepth_random_init',
 'experiments/EndoNerf cutting_baseline/cutting_baseline',
 'experiments/EndoNerf pulling_SurgeDepth_GRN/pulling_SurgeDepth_GRN',
 'experiments/SCARED_LONG_seq_2/SCARED_LONG_seq_2',
 'experiments/Stereomis_3/Stereomis_3',
 'experiments/EndoNerf pulling_SurgeDepth_random_init/pulling_SurgeDepth_random_init',
 'experiments/EndoNerf pulling_SurgeDepth_GRN_10_percent/pulling_SurgeDepth_GRN_10_percent',
 'experiments/EndoNerf cutting_SurgeDepth_base_10_percent/cutting_SurgeDepth_base_10_percent',
 'experiments/EndoNerf pulling_baseline/pulling_baseline',
 'experiments/2/2',
 'experiments/EndoNerf pulling_deform_simple_54_no_GRN_no_bloat/pullin

In [56]:
done = [ 'experiments/EndoNerf pulling_SurgeDepth_base_01_percent/pulling_SurgeDepth_base_01_percent','experiments/EndoNerf pulling_SurgeDepth_GRN_01_percent/pulling_SurgeDepth_GRN_01_percent','experiments/EndoNerf pulling_SurgeDepth_base_10_percent/pulling_SurgeDepth_base_10_percent','experiments/EndoNerf pulling_SurgeDepth_GRN_10_percent/pulling_SurgeDepth_GRN_10_percent','experiments/EndoNerf pulling_SurgeDepth_GRN/pulling_SurgeDepth_GRN','experiments/EndoNerf pulling_SurgeDepth_random_init/pulling_SurgeDepth_random_init','experiments/EndoNerf pulling_baseline/pulling_baseline','experiments/EndoNerf pulling_SurgeDepth_random_init/pulling_SurgeDepth_random_init', 'experiments/EndoNerf cutting_SurgeDepth_base_01_percent/cutting_SurgeDepth_base_01_percent', 'experiments/EndoNerf cutting_SurgeDepth_GRN_01_percent/cutting_SurgeDepth_GRN_01_percent','experiments/EndoNerf cutting_SurgeDepth_base_10_percent/cutting_SurgeDepth_base_10_percent','experiments/EndoNerf cutting_SurgeDepth_GRN_10_percent/cutting_SurgeDepth_GRN_10_percent','experiments/EndoNerf cutting_SurgeDepth/cutting_SurgeDepth','experiments/EndoNerf cutting_SurgeDepth_GRN/cutting_SurgeDepth_GRN','experiments/EndoNerf cutting_SurgeDepth_random_init/cutting_SurgeDepth_random_init','experiments/EndoNerf cutting_baseline/cutting_baseline']
todo = [image_path for image_path in image_paths if image_path not in done ]
todo

['experiments/SCARED_LONG_seq_2/SCARED_LONG_seq_2',
 'experiments/2/2',
 'experiments/EndoNerf pulling_SurgeDepth/pulling_SurgeDepth',
 'experiments/SCARED_LONG/SCARED_LONG']

In [7]:
import os
import numpy as np
i=5
experiment =   f'/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/Snellius_Exps/Stereomis_9_no_color_change_EndoDAC/Seq_9'
lpips_path = os.path.join(experiment, 'eval', 'lpips.txt')
psnr_path = os.path.join(experiment, 'eval', 'psnr.txt')
# rmse_path = os.path.join(experiment, 'eval', 'rmse.txt')
ssim_path = os.path.join(experiment, 'eval', 'ssim.txt')
nr_gauss_path = os.path.join(experiment, 'eval','nr_gaussians.txt')
depth_metrics_path = os.path.join(experiment,'eval','depth_errors.txt')
runtime_path = os.path.join(experiment, 'runtimes.txt')
lpips_scores = np.loadtxt(lpips_path)
psnr_scores = np.loadtxt(psnr_path)
ssim_scores = np.loadtxt(ssim_path)
nr_gaussians = np.loadtxt(nr_gauss_path)
depth_metrics = np.loadtxt(depth_metrics_path,delimiter=',')
len_dataset = len(nr_gaussians)
f = open(runtime_path, 'r')
# print(f'FPS: {1/float(f.read()[-22:-2])}')
fps = 1/float(f.read()[-21:-2])
f.close()

print("PSNR , SSIM , LPIPS&FPS&#Gaussians,len_dataset")
print("{:.4f}  &  {:.4f}  &  {:.4f} & {:.4f}& {:.0f} & {:.0f}".format(psnr_scores.mean(),ssim_scores.mean(),lpips_scores.mean(),fps,nr_gaussians.mean(),len_dataset))
# loaded_metrics = np.loadtxt('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/EndoNerf cutting_deform_simple_12_baseline_w_surgedepth_w_GRN_w_reduce_gaussians/cutting_deform_simple_12_baseline_w_surgedepth_w_GRN_w_reduce_gaussians/eval/lpips.txt')
# loaded_metrics
# lpips_scores.mean()
print('depth metrics: abs_rel & sq_rel & RMSE & RMSE_log & $\delta_1$& $\delta_2$& $\delta_2$& PSNR ')
print((8*"&     {:.4f}  ").format(*depth_metrics.mean(axis = 0)))

PSNR , SSIM , LPIPS&FPS&#Gaussians,len_dataset
23.0169  &  0.8313  &  0.1752 & 1.7926& 94508 & 124
depth metrics: abs_rel & sq_rel & RMSE & RMSE_log & $\delta_1$& $\delta_2$& $\delta_2$& PSNR 
&     0.1052  &     0.7811  &     8.0465  &     nan  &     0.8738  &     0.9775  &     0.9956  &     29.9083  


In [ ]:
# depth metrics: abs_rel & sq_rel & RMSE & RMSE_log & $\delta_1$& $\delta_2$& $\delta_2$& PSNR, SSIM 
# &     0.2439  &     4.6735  &     11.6724  &     0.2780  &     0.5342  &     0.9593  &     0.9806  &     20.7574 & 151
# &     0.2390  &     2.7043  &     9.0106  &     0.2798  &     0.5153  &     0.8947  &     0.9928  &     31.0911  & 63

# &     0.0118  &     0.0152  &     0.5699  &     0.0313  &     0.9971  &     0.9992  &     0.9997  &     33.9990  
# &     0.0201  &     0.0606  &     1.6411  &     0.0631  &     0.9936  &     0.9978  &     0.9991  &     39.0610  





abs_rel =  (0.0118  * 151 +
            0.0201* 63)/(151+63)
sq_rel = (  0.0152 * 151 +
            0.0606* 63)/(151+63)
RMSE = (0.5699 * 151 +
        1.6411* 63)/(151+63)

print("{:.4f}   &   {:.4f}   &  {:.4f}   ".format(abs_rel,sq_rel,RMSE))

0.0142   &   0.0286   &  0.8853   


In [8]:
# 34.5906  &  0.9763  &  0.0191 & 1.8846& 144967 & 112
# 30.3562  &  0.9621  &  0.0585 & 1.7286& 142031 & 125
# 36.6161  &  0.9846  &  0.0218 & 1.8244& 129884 & 125
# 23.7587  &  0.8347  &  0.1801 & 1.8891& 146993 & 124
# 34.3210  &  0.9782  &  0.0289 & 1.9045& 139041 & 124


# 31.9759  &  0.9491  &  0.0471 & 2.1415& 21771 & 112
# 29.3522  &  0.9547  &  0.0802 & 2.1010& 29990 & 125
# 35.3592  &  0.9795  &  0.0391 & 2.1527& 18906 & 125
# 21.0869  &  0.7115  &  0.2731 & 2.1466& 29798 & 124
# 32.4124  &  0.9671  &  0.0508 & 2.1300& 21407 & 124

# 37.0759  &  0.9920  &  0.0518 & 1.6436& 98595 & 19
# 34.6432  &  0.9792  &  0.0573 & 1.6617& 100540 & 7


# 27.6135  &  0.9175  &  0.1202 & 1.7154& 106572 & 112
# 21.9660  &  0.7679  &  0.2584 & 1.8153& 129227 & 125
# 29.7295  &  0.9416  &  0.0896 & 1.7559& 109618 & 125
# 20.3283  &  0.7580  &  0.2962 & 1.6698& 169451 & 124
# 23.0169  &  0.8313  &  0.1752 & 1.7926& 94508  & 124



PSNR = (        27.6135 * 112 + 
                21.9660 * 125 +
                29.7295 * 125 +
                20.3283 * 124 +
                23.0169 * 124 ) / (112+125+125+124+124)
SSIM = (        0.9175 * 112 + 
                0.7679 * 125 +
                0.9416 * 125 +
                0.7580 * 124 +
                0.8313 * 124 ) / (112+125+125+124+124)
LPIPS = (        0.1202* 112 + 
                0.2584* 125 +
                0.0896 * 125 +
                0.2962 * 124 +
                0.1752 * 124 ) / (112+125+125+124+124)
FPS = (        1.7154 * 112 + 
                1.8153 * 125 +
                1.7559 * 125 +
                1.6698 * 124 +
                1.7926 * 124 ) / (112+125+125+124+124)

print("PSNR , SSIM , LPIPS&FPS")
print("{:.4f}  &  {:.4f}  &  {:.4f} & {:.4f}".format(PSNR,SSIM,LPIPS,FPS))


PSNR , SSIM , LPIPS&FPS
24.4745  &  0.8418  &  0.1892 & 1.7506


In [ ]:
# import cv2
# import os
# base_path = '/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/data/SCARED_LONG/StereoMis/color'

# for path in os.listdir(base_path):
#     img = cv2.imread(os.path.join(base_path,path))
#     img_height = img.shape[0]
#     cropped_height = img_height//2
#     img_resized = img[:cropped_height,:]
#     cv2.imwrite(os.path.join(base_path,path),img_resized)
    

In [32]:
## no GRN,w/ Depth,rand init),100\%Gauss , 35.6458  ,  0.9789  ,  0.0673 , 1.5157, 115113 , 37.3969  ,  0.9912  ,  0.0338 , 1.6883, 117827\\
## no GRN,no Depth),100\%Gauss           , 33.1386  ,  0.9651  ,  0.0742 , 2.8108, 90645  , 36.2283  ,  0.9885  ,  0.0251 , 2.7306, 94526 \\
## no GRN, w/SD),100\%Gauss              , 35.6458  ,  0.9789  ,  0.0673 , 1.5157, 115113 , 38.8557  ,  0.9940  ,  0.0103 , 2.2322, 143256\\
## w/ GRN, w/SD),100\%Gauss              , 36.7405  ,  0.9820  ,  0.0355 , 2.0480, 108967 , 38.5460  ,  0.9931  ,  0.0162 , 2.1136, 114047\\
## w/ GRN, w/SD),10\%Gauss               , 36.1126  ,  0.9791  ,  0.0519 , 2.7710, 11813  , 37.6966  ,  0.9914  ,  0.0253 , 2.8073, 13753\\
## w/o GRN, w/SD),10\%Gauss              , 36.7637  ,  0.9833  ,  0.0317 , 2.6277, 41089  , 38.4466  ,  0.9931  ,  0.0130 , 2.6205, 48485\\
## w/ GRN, w/SD),1\%Gauss                , 34.2906  ,  0.9671  ,  0.1181 , 3.1629, 1894   , 34.7402  ,  0.9803  ,  0.1036 , 3.1959, 3017\\
## w/o GRN, w/SD),1\%Gauss               , 33.1964  ,  0.9349  ,  0.1038 , 3.0525, 13872  , 36.6134  ,  0.9732  ,  0.0462 , 2.9512, 19836\\

import numpy as np

a = np.array([33.1964  ,  0.9349  ,  0.1038 , 3.0525, 13872])*63
b = np.array([36.6134  ,  0.9732  ,  0.0462 , 2.9512, 19836])*156
# c = np.array([28.0204  ,  0.9190  ,  0.0710])*401
# d = np.array([30.8538  ,  0.9620  ,  0.0588])*326


avg = (a+b)/(63+156)
print('{:.2f}  & {:.2f}& {:.2f} & {:.2f}&{:.0f}'.format(avg[0],avg[1],avg[2],avg[3],avg[4]))

35.63  & 0.96& 0.06 & 2.98&18120


In [12]:
import numpy as np

cutting = np.loadtxt("/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/Snellius_Exps/EndoNerf cutting_SurgeDepth_GRN_EndoDAC/cutting_SurgeDepth_GRN_EndoDAC/eval/depth_errors.txt",delimiter=',')
pulling = np.loadtxt('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/Snellius_Exps/EndoNerf pulling_SurgeDepth_GRN_EndoDAC/pulling_SurgeDepth_GRN_EndoDAC/eval/depth_errors.txt',delimiter=',')


abs_rel = (pulling[:,0].mean()*len(pulling[:,0])+cutting[:,0].mean()*len(cutting[:,0]))/(pulling.shape[0]+cutting.shape[0])
sq_rel = (pulling[:,1].mean()*len(pulling[:,1])+cutting[:,1].mean()*len(cutting[:,1]))/(pulling.shape[0]+cutting.shape[0])
RMSE = (pulling[:,2].mean()*len(pulling[:,2])+cutting[:,2].mean()*len(cutting[:,2]))/(pulling.shape[0]+cutting.shape[0])

print("{:.4f}   &   {:.4f}   &  {:.4f}   ".format(abs_rel,sq_rel,RMSE))

0.1634   &   1.2918   &  5.5175   


In [10]:
pulling.shape[0]

7